# ECH GCP Private Service Connect Demo

## Install Prerequisites

In [ ]:
! pip install -q -U -r requirements.txt

## Create Environment

In [75]:
%%bash
terraform -chdir=terraform init  -upgrade
terraform -chdir=terraform apply -auto-approve

Initializing the backend...
Initializing provider plugins...
- Finding elastic/elasticstack versions matching "~> 0.11"...
- Finding elastic/ec versions matching "~> 0.9.0"...
- Finding hashicorp/google versions matching "~> 5.0"...
- Using previously-installed elastic/elasticstack v0.12.1
- Using previously-installed elastic/ec v0.9.0
- Using previously-installed hashicorp/google v5.45.2

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.
data.ec_stack.elastic_latest: Reading...
data.ec_stack.elastic_latest: Read complete after 1s [id=9.2.0]

Terraform used the selected providers to generate the following e

## Create Environment File

In [76]:
%%bash
cat > .env << EOF
ELASTIC_USERNAME=$(terraform -chdir=terraform output elastic_username)
ELASTIC_PASSWORD=$(terraform -chdir=terraform output elastic_password)
ELASTIC_INDEX_NAME=$(terraform -chdir=terraform output elastic_index_name)
ELASTIC_EXTERNAL_ENDPOINT=$(terraform -chdir=terraform output elastic_external_endpoint)
ELASTIC_PSC_ENDPOINT=$(terraform -chdir=terraform output elastic_psc_endpoint)
EOF

## Deploy Speed Test App to Central VM

In [77]:
%%bash
gcloud compute ssh central-vm --zone=us-central1-a --command="mkdir -p speedtest" --no-user-output-enabled && \
gcloud compute scp ./scripts/speed_test.py central-vm:~/speedtest --zone=us-central1-a --no-user-output-enabled && \
gcloud compute scp ./.env central-vm:~/speedtest --zone=us-central1-a --no-user-output-enabled && \
gcloud compute scp ./requirements.txt central-vm:~/speedtest --zone=us-central1-a --no-user-output-enabled && \
gcloud compute ssh central-vm --zone=us-central1-a --command="cd speedtest && python3 -m venv venv && source venv/bin/activate && pip install -q -U -r requirements.txt" --no-user-output-enabled && \
echo "Speed Test App deployed to Central VM"

Speed Test App deployed to Central VM


## Speed Test - Local Client to External Connection

In [78]:
! python3 scripts/speed_test.py external

*** Write Test ***

Latency Percentiles:
50th:     38.04 ms
95th:     54.59 ms
99th:     94.94 ms

*** Read Test ***

Latency Percentiles:
50th:     33.45 ms
95th:     47.46 ms
99th:     65.41 ms


## Speed Test - Client in same region as Private Connection

In [79]:
! gcloud compute ssh central-vm --zone=us-central1-a --command="cd speedtest && source venv/bin/activate && python3 speed_test.py psc" 2>/dev/null

*** Write Test ***

Latency Percentiles:
50th:      8.52 ms
95th:      9.90 ms
99th:     11.52 ms

*** Read Test ***

Latency Percentiles:
50th:      5.31 ms
95th:      8.45 ms
99th:     11.48 ms


## Destroy Environment

In [80]:
%%bash
terraform -chdir=terraform destroy -auto-approve
rm -f .env

data.ec_stack.elastic_latest: Reading...
google_compute_network.psc_vpc: Refreshing state... [id=projects/elastic-customer-eng/global/networks/psc-vpc]
data.ec_stack.elastic_latest: Read complete after 1s [id=9.2.0]
ec_deployment.primary: Refreshing state... [id=8fca7617926966fbd76a076491785ab1]
google_compute_subnetwork.psc_subnet: Refreshing state... [id=projects/elastic-customer-eng/regions/us-central1/subnetworks/psc-subnet]
google_compute_firewall.allow_ssh_vm: Refreshing state... [id=projects/elastic-customer-eng/global/firewalls/allow-ssh-vm]
google_dns_managed_zone.elastic_cluster_zone: Refreshing state... [id=projects/elastic-customer-eng/managedZones/elastic-cluster-zone]
google_compute_address.psc_endpoint: Refreshing state... [id=projects/elastic-customer-eng/regions/us-central1/addresses/psc-endpoint]
google_compute_instance.central_vm: Refreshing state... [id=projects/elastic-customer-eng/zones/us-central1-a/instances/central-vm]
google_dns_record_set.elastic_record_set: 